In [4]:
import os

# from the current `tute_1` directory, go back two levels to the `MAST30034` directory
output_relative_dir = '../data/landing/'

# check if it exists as it makedir will raise an error if it does exist
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)
    
# now, for each type of data set we will need, we will create the paths
for target_dir in ('tlc_data',): # taxi_zones should already exist
    if not os.path.exists(output_relative_dir + target_dir):
        os.makedirs(output_relative_dir + target_dir)

In [5]:
DATA_RANGE = {
    "2023": range(7, 13),
    "2024": range(1, 4),
}

# this is the URL template as of 07/2023
URL_TEMPLATE = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_"#year-month.parquet

In [6]:
from urllib.request import urlretrieve

# data output directory is `data/tlc_data/`
tlc_output_dir = output_relative_dir + 'tlc_data'

for year in DATA_RANGE:
    for month in DATA_RANGE[year]:
        # 0-fill i.e 1 -> 01, 2 -> 02, etc
        month = str(month).zfill(2) 
        print(f"Begin {month} {year}")
        
        # generate url
        url = f'{URL_TEMPLATE}{year}-{month}.parquet'
        # generate output location and filename
        output_dir = f"{tlc_output_dir}/{year}-{month}.parquet"
        # download
        urlretrieve(url, output_dir) 
        
        print(f"Completed {month} {year}")

Begin 07 2023
Completed 07 2023
Begin 08 2023
Completed 08 2023
Begin 09 2023
Completed 09 2023
Begin 10 2023
Completed 10 2023
Begin 11 2023
Completed 11 2023
Begin 12 2023
Completed 12 2023
Begin 01 2024
Completed 01 2024
Begin 02 2024
Completed 02 2024
Begin 03 2024
Completed 03 2024
